In [2]:
import pandas as pd

In [11]:
# Read file and process lines
with open('esawaai.csv', 'r') as f:
    lines = f.readlines()

# Remove leading/trailing whitespace and split on first space
data = []
for line in lines:
    line = line.strip()
    if line:  # Skip empty lines
        size, path = line.split(' ', 1)
        data.append([int(size), path])

# Convert to pandas DataFrame
df = pd.DataFrame(data, columns=['size', 'path'])

df

,size,path
0,1533285,esawaai/2019/08/26/S1A_IW_ESAWAAI__1SDV_201908...
1,185044694,esawaai/2019/08/26/S1A_IW_ESAWAAI__1SDV_201908...
2,168779279,esawaai/2019/08/26/S1A_IW_ESAWAAI__1SDV_201908...
3,144352380,esawaai/2019/08/27/S1A_IW_ESAWAAI__1SDV_201908...
4,209491051,esawaai/2019/08/27/S1A_IW_ESAWAAI__1SDV_201908...
...,...,...
18484,185157726,esawaai/2023/11/04/S1A_IW_ESAWAAI__1SDV_202311...
18485,169103739,esawaai/2023/11/04/S1A_IW_ESAWAAI__1SDV_202311...
18486,129979939,esawaai/2023/11/04/S1A_IW_ESAWAAI__1SDV_202311...
18487,184753718,esawaai/2023/11/04/S1A_IW_ESAWAAI__1SDV_202311...
